In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange


from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#
from tsnecuda import TSNE

# 
np.set_printoptions(suppress=True)


from seqnmf import seqnmf



Autosaving every 180 seconds


In [107]:
#####################################
######### LOAD CALCIUM DATA #########
#####################################
#
from sklearn import metrics
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

    
main_dir = '/media/cat/256GB/donato/'

animal_names = [
'DON-002865',
'DON-003165',
'DON-003343',
'DON-006084',
'DON-006085',
'DON-006087'
]

#animal_names = animal_names[:1]

trainings = [
'adult (8 month old) 19 sess pre wheel',
'adult (7 month old) 6 sess pre wheel',
'adult (6 month old) 0 before wheel',
'juvenile naive P19',
'juvenile naive P19',
'juvenile naive P24'
    
]


################################################
################################################
################################################ 
fig = plt.figure()
for ctr_animal_id, animal_name in enumerate(animal_names):

    #     
    root_dir = os.path.join(main_dir,
                            animal_name)

    # 
    sessions = sorted(os.listdir(root_dir))
    #print ("Sessions: ", sessions)

    remove_id = []
    for session in sessions:
        if 'tracking' in session:
            sessions.remove(session)
            break
    #print (sessions)
    # 

    #
    max_neurons = 0
    n_neurons = []
    aucs = []
    

    # 
    if False:
        fig = plt.figure()
        ax1 = plt.subplot(121)
        ax2 = plt.subplot(122)
        ax3 = ax2.twinx()
        
        ax1.set_ylim(0,1)
        ax1.set_xlim(0,1)
        ax1.legend()
        ax1.set_xlabel("# Pcs (normalized to # cells)",fontsize=30)
        ax1.set_ylabel("Cumulative explained variance",fontsize=30)
        ax1.tick_params(axis='both', which='both', labelsize=30)


    else:
        ax2 = plt.subplot(2,3,ctr_animal_id+1)
        ax3 = ax2.twinx()
    
    # 
    cmap = plt.get_cmap("jet", len(sessions))

    #
    for ctr, session in enumerate(sessions):

        # The data can have 3 diff structures;
        session_dir = os.path.join(root_dir, session,'suite2p','plane0')

        c = calcium.Calcium()
        c.root_dir = session_dir

        # run PCA
        pca, X_pca = c.compute_PCA(None)
        n_neurons.append(X_pca.shape[1])

        # check this to set plot limits
        if X_pca.shape[1]>max_neurons:
            max_neurons = X_pca.shape[1]
            
        # run tsne
        var_exp = pca.explained_variance_ratio_
        d = np.arange(var_exp.shape[0])/var_exp.shape[0]
        
        # 
        if False:
            ax1.plot(d,np.cumsum(var_exp),
                    c=cmap(ctr), label = session)            

        #
        ax2.scatter(ctr, X_pca.shape[1],
                    s=200,
                    color=cmap(ctr))

        #        
        auc = metrics.auc(d,np.cumsum(var_exp))
        ax3.scatter(ctr, auc,
                   marker='^',
                   s=200,
                   color=cmap(ctr))
        aucs.append(auc)

    ######################################################################
    ########################## FIT AUC CURVES ############################
    ######################################################################
    regr_auc = linear_model.LinearRegression()

    # Train the model using the training sets
    x = np.arange(len(aucs)).reshape(-1,1)
    y = np.array(aucs).reshape(-1,1)
    regr_auc.fit(x, y)

    # Make predictions using the testing set
    pred_y = regr_auc.predict(x)

    ax3.plot(x, pred_y,
             c='black',
             linewidth=3)
    ######################################################################
    ##################### FIT # NEURON CURVES ############################
    ######################################################################
    regr_neurons = linear_model.LinearRegression()

    # Train the model using the training sets
    x = np.arange(len(n_neurons)).reshape(-1,1)
    y = np.array(n_neurons).reshape(-1,1)
    print (x.shape, y.shape)
    regr_neurons.fit(x, y)

    # Make predictions using the testing set
    pred_y = regr_neurons.predict(x)

    ax2.plot(x, pred_y,
                          '--',c='black',
             linewidth=3
            )
    
    
    
    ######################################################################
    ####################### CLEAN UP PLOTS ###############################
    ######################################################################


    # 
    if False:
        fontsize=30
        plt.suptitle(animal_name + ", " + trainings[ctr_animal_id], fontsize=fontsize)
    else:
        fontsize=10
        plt.title(animal_name + ", " + trainings[ctr_animal_id]+ 
                  ", auc slope "+str(round(regr_auc.coef_[0][0],4))+
                  ", n_neurons slope "+str(round(regr_neurons.coef_[0][0],4))
                                    , fontsize=fontsize)
        
    ax2.set_ylim(0, max_neurons*1.5)
    ax2.set_xlim(0,len(sessions))
    ax2.set_xlabel("Session Day (chronological)",fontsize=fontsize)
    ax2.set_ylabel("CIRCLES - # of detected cells (suite2p uncleaned)",fontsize=fontsize)
    ax2.tick_params(axis='both', which='both', labelsize=fontsize)

    #
    #ax3.set_xticks()
    ax3.tick_params(axis='both', which='both', labelsize=fontsize)
    ax3.set_ylim(0,1)
    ax3.set_ylabel("TRIANGLES - Area under the variance explained curve ",fontsize=fontsize)
    #
plt.show()


(22, 1) (22, 1)
(18, 1) (18, 1)
(20, 1) (20, 1)
(10, 1) (10, 1)
(10, 1) (10, 1)
(10, 1) (10, 1)


In [51]:
import numpy as np
plt.show()


0.375


In [6]:
########################################
########################################
########################################
def find_sequences(data, thresh=1):
    
    #
    segs = []
    seg = []
    seg.append(0)
    
    clrs = []
    ctr=0
    clrs.append(ctr)
    
    # 
    for k in trange(1,data.shape[0],1):
        
        temp = dist = np.linalg.norm(data[k]-data[k-1])
        
        if temp<=thresh:
            seg.append(k)
            clrs.append(ctr)
        else:
            segs.append(seg)
            seg = []
            seg.append(k)
            
            
            #
            ctr = np.random.randint(10000)
            clrs.append(ctr)
        
    # add last segment if missed:
    if len(segs[-1])>1:
        segs.append(seg)

    return segs, clrs

############################
############################
############################
thresh = 1

X_mode = X_tsne
segs, clrs = find_sequences(
                           #X_umap,
                           X_mode,
                           thresh)

print ("# segs: ", len(segs), X_mode.shape)
clrs = np.array(clrs)
print ("Clrs: ", clrs.shape)
#
fig=plt.figure()
plt.scatter(X_mode[:5000,0],
            X_mode[:5000,1],
            c=clrs[:5000],
           cmap='Paired')


plt.show()


NameError: name 'X_tsne' is not defined

In [7]:
############################################
############################################
############################################
v = visualize.Visualize()

############################################
title='velocity (m/s)'
cmap = 'electric'

v.plot_scatter_3d( 
                   X_mode[:,:2], 
                   clrs,
                   title,
                   cmap,
                   1,1
                  )



NameError: name 'X_mode' is not defined

In [133]:
#############################################
#############################################
#############################################

sums = 0
lens = []
# 
for s in range(len(segs)):
    lens.append(len(segs[s]))

width = 5
bins = np.arange(0,500,width)

plt.figure()
y = np.histogram(lens,bins)  
t = (y[1][:-1]+width/2.)/30
plt.plot(t, y[0])
plt.xlabel("Duration of sequence (sec)")
plt.ylabel("# of sequences")
plt.xlim(t[0],t[-1])
plt.ylim(0,np.max(y[0]))
plt.show()
    
    

In [64]:
############################################
############################################
############################################
from sklearn.manifold import Isomap

#
embedding = Isomap(n_components=3)

#
X_isomap = embedding.fit_transform(X[::10])
print (X_isomap.shape)



(5574, 3)


In [68]:
############################################
############################################
############################################
v = visualize.Visualize()

############################################
title='velocity (m/s)'
cmap = 'viridis'

v.plot_scatter_3d( 
                   X_isomap[:,:3], 
                 w.track.velocity.values[::10],
                 title,
                 cmap,
                   1,1
                  )


#############################################
title='frame'
cmap = 'magenta_r'
v.plot_scatter_3d( 
             X_isomap[:,:3], 
             None,
             title,
             cmap,
            1,1)



In [69]:
############################################
############################################
############################################

from laplacian_eigenmaps import LE

#
le = LE.LE(X[::10], dim = 2, k = 3, 
           graph = 'k-nearest', 
           weights = 'heat kernel', 
           sigma = 5, 
           laplacian = 'symmetrized')

# 
X_LE = le.transform()
print(X_LE.shape)



/home/cat/code/manifolds/laplacian_eigenmaps/LE.py:137: UserWarning:

Graph is not fully connected, Laplacian Eigenmaps may not work as expected

/home/cat/code/manifolds/laplacian_eigenmaps/LE.py:137: UserWarning:

Graph is not fully connected, Laplacian Eigenmaps may not work as expected



(5574, 3)


In [72]:
############################################
############################################
############################################
v = visualize.Visualize()

############################################
title='velocity (m/s)'
cmap = 'viridis'

v.plot_scatter_3d( 
                   np.real(Y[:,:3]), 
                 w.track.velocity.values[::10],
                 title,
                 cmap,
                   1,1
                  )


#############################################
title='frame'
cmap = 'magenta_r'
v.plot_scatter_3d( 
                   np.real(Y[:,:3]), 
             None,
             title,
             cmap,
            1,1)